# Import Libary

In [1]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load Dataset

In [2]:
df = pd.read_csv('GrammarandProductReviewsupdate.csv')
df.head()

,Unnamed: 0,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,...,reviews_doRecommend,reviews_id,reviews_numHelpful,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_username,upc,Review
0,0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,...,NaN,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Joshua,6.02537E+11,awesome love album good hip hop side current p...
1,1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,...,NaN,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,Dorothy W,73416000391,good good flavor review collect part promotion
2,2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,...,NaN,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,Dorothy W,73416000391,good good flavor
3,3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,...,False,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,Rebecca,67981934427,disappoint read review look buy one couple lub...
4,4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,...,False,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,Walker557,67981934427,irritation husband buy gel us gel caused irrit...


In [3]:
df['reviews_rating'].value_counts()

5    46477
4    14583
3     4359
1     3697
2     1825
Name: reviews_rating, dtype: int64

there is 5 reviews_rating  in dataset so we convert those into 2 reviews means postive (4,5) and negative(1,2,3)

In [4]:
df['reviews_rating'][df['reviews_rating'] <=3] = 0

C:\Users\PAVILION\AppData\Local\Temp\ipykernel_3404\137808498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews_rating'][df['reviews_rating'] <=3] = 0


In [5]:
df['reviews_rating'][df['reviews_rating'] >= 4] = 1

C:\Users\PAVILION\AppData\Local\Temp\ipykernel_3404\1183928658.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews_rating'][df['reviews_rating'] >= 4] = 1


In [6]:
df['reviews_rating'].value_counts()

1    61060
0     9881
Name: reviews_rating, dtype: int64

# Smote Method

In [7]:
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

In [8]:
# separate both type variable in target values
label_0 = df[df['reviews_rating'] == 0]
label_1 = df[df['reviews_rating'] == 1]

In [9]:
print('Default values:',label_0.shape)
print('Non Default values:',label_1.shape)

Default values: (9881, 25)
Non Default values: (61060, 25)


In [10]:
# now apply Downsampling for less data
y= resample(label_0, replace=True, n_samples=len(label_1),random_state=45)

In [11]:
y.shape

(61060, 25)

In [12]:
ds=pd.concat([y,label_1])  # merge both dataset value

In [13]:
ds['reviews_rating'].value_counts()

0    61060
1    61060
Name: reviews_rating, dtype: int64

In [14]:
label0=ds[ds['reviews_rating'] == 0].head(20000)
label1=ds[ds['reviews_rating'] == 1].head(20000)

In [15]:
df1= pd.concat([label1,label0])  # merge both dataset value

In [16]:
df1.shape

(40000, 25)

In [17]:
tokenizer = Tokenizer(num_words=500, split=' ')
tokenizer.fit_on_texts(df1['Review'].values)
x=tokenizer.texts_to_sequences(df1['Review'].values)
x = pad_sequences(x)
x

array([[  0,   0,   0, ...,  78, 250,  69],
       [  0,   0,   0, ...,   9,   6,   8],
       [  0,   0,   0, ...,   7,   7, 470],
       ...,
       [  0,   0,   0, ...,   1,  15, 141],
       [  0,   0,   0, ...,   1,   7,  14],
       [  0,   0,   0, ...,  30, 135, 107]])

In [18]:
model= Sequential()
model.add(Embedding(500,120, input_length=x.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 272, 120)          60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 272, 120)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 176)               209088    
                                                                 
 dense (Dense)               (None, 2)                 354       
                                                                 
Total params: 269,442
Trainable params: 269,442
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
#splitting the data into training and testing
y = pd.get_dummies(df1['reviews_rating'])
x_train,x_test, y_train,y_test= train_test_split(x, y, test_size=0.3, random_state=42)

In [20]:
x_train= np.array(x_train)
x_test = np.array(x_test)
y_train= np.array(y_train)
y_test = np.array(y_test)

In [21]:
model1 = model.fit(x_train,y_train, epochs=5, batch_size=32, verbose=1,validation_data=(x_test,y_test))
model1

Epoch 1/5
875/875 [==============================] - 893s 1s/step - loss: 0.2891 - accuracy: 0.8781 - val_loss: 0.2382 - val_accuracy: 0.9038
Epoch 2/5
875/875 [==============================] - 1592s 2s/step - loss: 0.2362 - accuracy: 0.9049 - val_loss: 0.2157 - val_accuracy: 0.9153
Epoch 3/5
875/875 [==============================] - 2725s 3s/step - loss: 0.2189 - accuracy: 0.9102 - val_loss: 0.2112 - val_accuracy: 0.9158
Epoch 4/5
875/875 [==============================] - 1432s 2s/step - loss: 0.2097 - accuracy: 0.9143 - val_loss: 0.2102 - val_accuracy: 0.9158
Epoch 5/5
875/875 [==============================] - 938s 1s/step - loss: 0.1990 - accuracy: 0.9187 - val_loss: 0.2094 - val_accuracy: 0.9172


In [22]:
model.evaluate(x_test,y_test)

375/375 [==============================] - 36s 95ms/step - loss: 0.2094 - accuracy: 0.9172


[0.20936362445354462, 0.9171666502952576]

In [23]:
model.save('lstm_model.h5')